# Description

In this notebook, we will build a custom quadratic layer which computes y = ax^2 + bx + c.

# Imports

In [1]:
import numpy as np

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.layers import Layer, Input, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

# Cusotm Layer

In [2]:
class CustomLayer(Layer):
    
    def __init__(self, units=1, activation=None):
        super(CustomLayer, self).__init__()
        self.units=units
        self.activation=tf.keras.activations.get(activation)  
        
    def build(self, input_shape):
        w_init=tf.random_normal_initializer()
        self.a=tf.Variable(name='a',
                           initial_value=w_init(shape=(input_shape[-1], self.units), dtype="float32"),
                           trainable=True
                          )
        self.b=tf.Variable(name='b',
                           initial_value=w_init(shape=(input_shape[-1], self.units), dtype="float32"),
                           trainable=True
                          )
        c_init=tf.zeros_initializer()
        self.c=tf.Variable(name="c",
                          initial_value=c_init(shape=(self.units,), dtype="float32"),
                          trainable=True
                          )
    def call(self, inputs):
        return self.activation(tf.matmul( K.square(inputs), self.a) + tf.matmul(inputs, self.b) + self.c)


# Data

In [8]:
(x_train, y_train), (x_test, y_test)=mnist.load_data()
x_train= x_train / 255.0
x_test=x_test / 255.0

# Model

In [4]:
input_layer=Input(shape=(28,28))
x=Flatten()(input_layer)
x=CustomLayer(units=128, activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(10, activation='softmax')(x)
model=Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
history=model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2670 - accuracy: 0.9215
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1306 - accuracy: 0.9604
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0999 - accuracy: 0.9690
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0824 - accuracy: 0.9743
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0728 - accuracy: 0.9770


In [9]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 1ms/step - loss: 0.0859 - accuracy: 0.9755


[0.0858626589179039, 0.9754999876022339]